In [ ]:

import os

import numpy as np
import pandas as pd
import requests


def get_routes(osm_id: int, public_transport_type: str):
    overpass_url = os.environ.get("OVERPASS_URL", "http://lz4.overpass-api.de/api/interpreter")
    overpass_query = f"""
    [out:json];
            (
                relation({osm_id});
            );map_to_area;
            (
                relation(area)['route'='{public_transport_type}'];
            );
    out geom;
    """
    result = requests.get(overpass_url, params={"data": overpass_query})
    json_result = result.json()["elements"]

    return pd.DataFrame(json_result)

In [ ]:
test = pd.DataFrame(data_osm.iloc[1]['members'])
test = test[test['type'] == 'node']

In [90]:
osm_id = 4869409
public_transport_type = "bus"
data_osm = get_routes(osm_id, public_transport_type)

In [113]:
from scipy.spatial import cKDTree
from shapely import LineString, Point
import geopandas as gpd

PLATFORM_ROLES = ['platform_entry_only', 'platform', 'platform_exit_only']


def parse_overpass_route_response(loc: dict):
    if 'ref' in loc['tags'].keys():
        transport_name = loc['tags']['ref']
    elif 'name' in loc['tags'].keys():
        transport_name = loc['tags']['name']
    else:
        transport_name = None

    route = pd.DataFrame(loc["members"])

    if "node" in route["type"].unique():
        stops = route[(route['type'] == 'node') & (route['role'].isin(PLATFORM_ROLES))]
        stops['geometry'] = stops.apply(lambda x: (Point(x["lon"], x["lat"])), axis=1)
        stops.reset_index(drop=True, inplace=True)
    else:
        stops = None

    ways = route[route["type"] == "way"]

    if len(ways) > 0:
        ways = ways["geometry"].reset_index(drop=True)
        ways = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()

        connected_ways = [[]]
        cur_way = 0
        for coords in ways:
            if not connected_ways[cur_way]:
                connected_ways[cur_way] += coords
                continue
            if coords[0] == coords[-1]:
                continue
            if connected_ways[cur_way][-1] == coords[0]:
                connected_ways[cur_way] += coords[1:]

            elif connected_ways[cur_way][-1] == coords[-1]:
                connected_ways[cur_way] += coords[::-1][1:]

            elif connected_ways[cur_way][0] == coords[0]:
                connected_ways[cur_way] = coords[1:][::-1] + connected_ways[cur_way]

            elif connected_ways[cur_way][0] == coords[-1]:
                connected_ways[cur_way] = coords + connected_ways[cur_way][1:]

            else:
                connected_ways += [coords]
                cur_way += 1

        if len(connected_ways) > 1:
            for coords in connected_ways:
                print(coords[0], coords[-1])
    else:
        connected_ways = None

    return pd.Series({"way": connected_ways, "platforms": stops, 'name': transport_name})


data_geom = data_osm.apply(parse_overpass_route_response, axis=1, result_type='expand')

(46.0922111, 51.4961684) (46.0625728, 51.528984)
(46.125136, 51.498949) (46.0924441, 51.4959176)
(46.1271524, 51.5008024) (46.1257548, 51.4998442)
(46.0624318, 51.5291964) (46.0008441, 51.5405848)
(46.0008441, 51.5405848) (46.1013639, 51.4860569)
(46.1019604, 51.4861853) (46.1027033, 51.4861481)
(46.1030871, 51.4861757) (46.125136, 51.498949)
(46.1271524, 51.5008024) (46.1257548, 51.4998442)


In [162]:
import numpy as np
from scipy.spatial.distance import cdist

connected_ways = data_geom.iloc[2]['way']
connect_points = [point for coords in connected_ways for point in (coords[0], coords[-1])]

# connect_points = np.array(connect_points)
distances = cdist(connect_points, connect_points)
n = distances.shape[0]
mask = (np.arange(n)[:, None]//2) == (np.arange(n)//2)
distances[mask] = np.inf

for i in range(len(connected_ways)-1):
    min_index = np.unravel_index(np.argmin(distances), distances.shape)
    distances[min_index[0], :] = np.inf
    distances[min_index[1], :] = np.inf
    distances[:, min_index[0]] = np.inf
    distances[:, min_index[1]] = np.inf
    print(min_index)


(3, 4)
(1, 2)
(5, 7)


In [154]:
## Теперь соединяем жостко точки бам бам бим бим, исопльзуем % чтоб понять шо где начало или конец

array([[       inf, 0.11435699, 0.11482078, 0.11549308, 0.11581874,
        0.13108019, 0.13242517, 0.13138676],
       [0.11435699,        inf, 0.00061016, 0.0013425 , 0.00172729,
        0.02704291, 0.02970651, 0.02801795],
       [0.11482078, 0.00061016,        inf, 0.00074383, 0.00112674,
        0.0264579 , 0.02912553, 0.02743609],
       [0.11549308, 0.0013425 , 0.00074383,        inf, 0.00038479,
        0.02582807, 0.02850451, 0.02681333],
       [0.11581874, 0.00172729, 0.00112674, 0.00038479,        inf,
        0.02548159, 0.02816166, 0.02646984],
       [0.13108019, 0.02704291, 0.0264579 , 0.02582807, 0.02548159,
               inf, 0.00273879, 0.00108825],
       [0.13242517, 0.02970651, 0.02912553, 0.02850451, 0.02816166,
        0.00273879,        inf, 0.00169453],
       [0.13138676, 0.02801795, 0.02743609, 0.02681333, 0.02646984,
        0.00108825, 0.00169453,        inf]])

In [95]:
data_geom.iloc[1]['platforms']

,type,ref,role,lat,lon,geometry
0,node,1300550439,platform_entry_only,51.500745,46.127004,POINT (46.1270045 51.5007453)
1,node,11111670849,platform,51.492941,46.119462,POINT (46.1194617 51.4929409)
2,node,1100197348,platform,51.496383,46.115406,POINT (46.1154056 51.496383)
3,node,11111670852,platform,51.496772,46.113979,POINT (46.1139785 51.4967723)
4,node,11111670853,platform,51.491671,46.110916,POINT (46.1109159 51.4916707)
5,node,11111670854,platform,51.489596,46.108851,POINT (46.1088508 51.4895965)
6,node,11111670855,platform,51.486515,46.106265,POINT (46.1062654 51.4865151)
7,node,2040903107,platform,51.497763,46.089539,POINT (46.0895391 51.4977629)
8,node,1370767399,platform,51.500173,46.085037,POINT (46.0850367 51.5001727)
9,node,1370767395,platform,51.502840,46.080503,POINT (46.0805027 51.5028404)


In [100]:

m1 = gpd.GeoDataFrame(geometry=[LineString(x) for x in data_geom.iloc[1]['way']], crs=4326).reset_index().explore()
gpd.GeoDataFrame(data_geom.iloc[1]['platforms'], crs=4326).reset_index().explore(m=m1, color='red')

In [ ]:
test = pd.DataFrame(data_osm.iloc[2]['members'])
test = test[(test['type'] == 'node') & (test['role'].isin(['platform_entry_only', 'platform', 'platform_exit_only']))]
test['geometry'] = test.apply(lambda x: (Point(x["lon"], x["lat"])), axis=1)
test.reset_index(drop=True, inplace=True)
gdf = gpd.GeoDataFrame(test.reset_index(), crs=4326)
gdf.geometry = gdf.geometry.buffer(0.0001)
gdf.explore()

In [ ]:
test = pd.DataFrame(data_osm.iloc[1]['members'])
test = test[test['type'] == 'way']
test = test["geometry"]
test = test.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x]))
# gdf = gpd.GeoDataFrame(test.reset_index(),geometry='geometry',crs=4326)
# gdf.geometry = gdf.geometry.buffer(0.0001)
# gdf.explore()
test

In [ ]:
test

#### ПОПЫТКА СОЕДИНЯТЬ ДОРОГИ 

In [ ]:
from scipy.spatial import cKDTree
from shapely import LineString, Point
import geopandas as gpd

PLATFORM_ROLES = []


def parse_overpass_route_response(loc: dict):
    if 'ref' in loc['tags'].keys():
        transport_name = loc['tags']['ref']
    else:
        transport_name = loc['tags']['name']
    route = pd.DataFrame(loc["members"])

    ways = route[route["type"] == "way"]

    if len(ways) > 0:
        ways = ways["geometry"].reset_index(drop=True)
        ways = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()
        if ways[0][0] == ways[1][0] or ways[0][0] == ways[1][-1]:
            complete_line = ways[0][::-1]
        else:
            complete_line = ways[0]
        for coords in ways[1:]:
            if complete_line[-1] == coords[0]:
                complete_line += coords[1:]
            elif complete_line[-1] == coords[-1]:
                complete_line += coords[::-1][1:]
            else:
                if coords[0] == coords[-1]:
                    continue
                else:
                    tree = cKDTree([coords[0], coords[-1]])
                    _, ind = tree.query(complete_line[-1])
                    if ind == 1:
                        coords = coords[::-1]
                    complete_line += coords
        # print('^^^^^^^^^^')

    else:
        complete_line = None
    if "node" in route["type"].unique():
        platforms = route[(route["type"] == "node")].apply(lambda x: Point(x["lon"], x["lat"]), axis=1)
    else:
        platforms = None

    return pd.Series({"way": complete_line, "platforms": platforms, 'name': transport_name})


data_geom = data_osm.apply(parse_overpass_route_response, axis=1, result_type='expand')
data_geom

In [ ]:
test = [LineString(
    [(46.0046822, 51.5426408), (46.0046214, 51.5425402), (46.0042506, 51.541918), (46.0038134, 51.5411845),
     (46.0036437, 51.5408997), (46.0035859, 51.5408011), (46.0035352, 51.5407158), (46.0031053, 51.5399929),
     (46.0028598, 51.5395801), (46.0026303, 51.5391941), (46.0025706, 51.5390938), (46.0023997, 51.5391331),
     (46.00211, 51.5391985), (46.0019414, 51.5392372), (46.0008809, 51.5394906), (46.00081, 51.5395332),
     (46.000665, 51.5396234), (46.0006024, 51.5396705), (46.0005255, 51.5397869), (46.000459, 51.5399285),
     (46.0005332, 51.54006), (46.0007185, 51.5403886), (46.0008003, 51.5405164), (46.0008441, 51.5405848)]), LineString(
    [(46.0046822, 51.5426408), (46.0048226, 51.5426046), (46.0048853, 51.5425884), (46.0065015, 51.5422214),
     (46.0067008, 51.5421762), (46.007038, 51.5420996), (46.0071807, 51.5420672)]), Point(46.0046822, 51.5426408)]

gpd.GeoDataFrame(geometry=test, crs=4326).explore()

In [ ]:
gpd.GeoDataFrame(geometry=data_geom['way'].apply(LineString), crs=4326).reset_index().explore(column='index')

In [ ]:
print(data_geom['way'].apply(LineString).iloc[1])

In [ ]:
data_geom.iloc[16]['way']

In [ ]:
ways = data_geom.iloc[16]['way']
ways = ways["geometry"].reset_index(drop=True)
lines = ways.apply(lambda x: ([(coords["lon"], coords["lat"]) for coords in x])).tolist()
if lines[0][0] == lines[1][0]:
    complete_line = lines[0][::-1]
else:
    complete_line = lines[0]
for coords in lines[1:]:
    if complete_line[-1] == coords[0]:
        complete_line += coords[1:]
    elif complete_line[-1] == coords[-1]:
        complete_line += coords[::-1][1:]
    else:
        print('\n\nWTF\n\n')



In [ ]:
gpd.GeoDataFrame(geometry=[LineString(complete_line)], crs=4326).explore()

In [ ]:

from shapely import MultiPoint

gpd.GeoDataFrame(data=[{'kek': i} for i in data_geom.index.tolist()],
                 geometry=data_geom['platforms'].apply(lambda x: MultiPoint(x)), crs=4326).explore(column='kek')

In [ ]:

data_geom.index.tolist()